In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
import matplotlib.pyplot as plt
import numpy as np
import tfcoreml
import pandas as pd
from sklearn import preprocessing

In [2]:
dataframe = pd.read_csv("../data/housing.csv", delim_whitespace=True, header=None)
dataframe.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
0     506 non-null float64
1     506 non-null float64
2     506 non-null float64
3     506 non-null int64
4     506 non-null float64
5     506 non-null float64
6     506 non-null float64
7     506 non-null float64
8     506 non-null int64
9     506 non-null float64
10    506 non-null float64
11    506 non-null float64
12    506 non-null float64
13    506 non-null float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [4]:
dataframe.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64

In [5]:
dataframe.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [6]:
train_dataset = dataframe.sample(frac=0.8,random_state=0)
test_dataset = dataframe.drop(train_dataset.index)

train_dataset.shape, test_dataset.shape

((405, 14), (101, 14))

In [7]:
train_stats = train_dataset.describe()
train_stats.pop(13)
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
0,405.0,3.670339,8.977954,0.00632,0.08265,0.26838,3.47428,88.9762
1,405.0,11.067901,22.688396,0.00000,0.00000,0.00000,12.50000,95.0000
2,405.0,11.097951,6.699157,0.74000,5.32000,9.69000,18.10000,27.7400
3,405.0,0.069136,0.253999,0.00000,0.00000,0.00000,0.00000,1.0000
4,405.0,0.553098,0.115564,0.38500,0.44900,0.53800,0.62400,0.8710
5,405.0,6.269768,0.690552,3.56100,5.87800,6.19300,6.61900,8.7800
6,405.0,68.241975,27.908527,2.90000,45.00000,76.50000,94.10000,100.0000
7,405.0,3.834098,2.162173,1.12960,2.07880,3.27210,5.21460,12.1265
8,405.0,9.474074,8.618564,1.00000,4.00000,5.00000,24.00000,24.0000
9,405.0,405.839506,164.057951,187.00000,284.00000,334.00000,666.00000,711.0000


In [8]:
train_labels = train_dataset.pop(13)
test_labels = test_dataset.pop(13)

train_dataset.shape, test_dataset.shape, train_labels.shape, test_labels.shape

((405, 13), (101, 13), (405,), (101,))

In [9]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

normed_train_data.shape, normed_test_data.shape, train_labels.shape, test_labels.shape

((405, 13), (101, 13), (405,), (101,))

In [10]:
normed_train_data.to_csv("../data/normed_train_data.csv", index = None, header=False)
normed_test_data.to_csv("../data/normed_test_data.csv", index = None, header=False)
train_labels.to_csv("../data/train_labels.csv", index = None, header=False)
test_labels.to_csv("../data/test_labels.csv", index = None, header=False)

In [11]:
train = normed_train_data.join(train_labels)
test = normed_test_data.join(test_labels)

train.to_csv("../data/train.csv", index = None, header=False)
test.to_csv("../data/test.csv", index = None, header=False)

In [12]:
X = normed_train_data.values
Y = train_labels.values

In [13]:
keras_model = Sequential([
    Dense(64, input_shape=[13], activation='relu'),
    Dense(32, activation='relu'),
    Dense(1),
])

In [14]:
optimizer = tf.keras.optimizers.RMSprop(0.001)  #Adam()

keras_model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])

In [15]:
keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 3,009
Trainable params: 3,009
Non-trainable params: 0
_________________________________________________________________


In [16]:
keras_model.save('../model/keras_no_train_model.h5')

# get input, output node names for the TF graph from the Keras model
input_name = keras_model.inputs[0].name.split(':')[0]
keras_output_node_name = keras_model.outputs[0].name.split(':')[0]
graph_output_node_name = keras_output_node_name.split('/')[-1]

print(input_name)
print(keras_output_node_name)
print(graph_output_node_name)

core_model = tfcoreml.convert(tf_model_path='../model/keras_no_train_model.h5',
                         input_name_shape_dict={input_name: (1, 13)},
                         output_feature_names=[graph_output_node_name],
                         minimum_ios_deployment_target='13')
core_model.save('../model/keras_no_train_model.mlmodel')

dense_input
dense_2/Identity
Identity
0 assert nodes deleted
['sequential/dense_2/MatMul/ReadVariableOp/resource:0', 'sequential/dense/MatMul/ReadVariableOp/resource:0', 'sequential/dense/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense_2/MatMul/ReadVariableOp:0', 'sequential/dense_2/BiasAdd/ReadVariableOp:0', 'sequential/dense/MatMul/ReadVariableOp:0', 'sequential/dense_1/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense_1/BiasAdd/ReadVariableOp:0', 'sequential/dense_2/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense_1/MatMul/ReadVariableOp:0', 'sequential/dense/BiasAdd/ReadVariableOp:0', 'sequential/dense_1/MatMul/ReadVariableOp/resource:0']
6 nodes deleted
0 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 6 nodes.
2 identity nodes deleted
3 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/6] Converting op type: 'Placeholder', name: 'dense_input', output_shape: (1, 13).
[SSAConverter] [2/6] Converting op type: 'M

In [17]:
history = keras_model.fit(X, Y, epochs=500, verbose=1)

Train on 405 samples
Epoch 1/500
405/405 [==============================] - 1s 1ms/sample - loss: 558.0246 - mae: 21.9454 - mse: 558.0246
Epoch 2/500
405/405 [==============================] - 0s 39us/sample - loss: 503.6883 - mae: 20.7222 - mse: 503.6882
Epoch 3/500
405/405 [==============================] - 0s 37us/sample - loss: 442.4639 - mae: 19.2979 - mse: 442.4639
Epoch 4/500
405/405 [==============================] - 0s 37us/sample - loss: 372.7287 - mae: 17.5371 - mse: 372.7287
Epoch 5/500
405/405 [==============================] - 0s 37us/sample - loss: 297.7672 - mae: 15.4228 - mse: 297.7672
Epoch 6/500
405/405 [==============================] - 0s 37us/sample - loss: 223.5237 - mae: 12.9528 - mse: 223.5237
Epoch 7/500
405/405 [==============================] - 0s 37us/sample - loss: 158.3328 - mae: 10.3922 - mse: 158.3328
Epoch 8/500
405/405 [==============================] - 0s 37us/sample - loss: 108.0136 - mae: 8.2130 - mse: 108.0136
Epoch 9/500
405/405 [================

405/405 [==============================] - 0s 34us/sample - loss: 9.9292 - mae: 2.1626 - mse: 9.9292
Epoch 72/500
405/405 [==============================] - 0s 35us/sample - loss: 10.0397 - mae: 2.2117 - mse: 10.0397
Epoch 73/500
405/405 [==============================] - 0s 36us/sample - loss: 9.8282 - mae: 2.1664 - mse: 9.8282
Epoch 74/500
405/405 [==============================] - 0s 35us/sample - loss: 9.9732 - mae: 2.1659 - mse: 9.9732
Epoch 75/500
405/405 [==============================] - 0s 34us/sample - loss: 9.9129 - mae: 2.1373 - mse: 9.9129
Epoch 76/500
405/405 [==============================] - 0s 34us/sample - loss: 9.6162 - mae: 2.1365 - mse: 9.6162
Epoch 77/500
405/405 [==============================] - 0s 34us/sample - loss: 9.8205 - mae: 2.1686 - mse: 9.8205
Epoch 78/500
405/405 [==============================] - 0s 35us/sample - loss: 9.6608 - mae: 2.1478 - mse: 9.6608
Epoch 79/500
405/405 [==============================] - 0s 34us/sample - loss: 9.6843 - mae: 2.1528

405/405 [==============================] - 0s 34us/sample - loss: 7.0103 - mae: 1.8768 - mse: 7.0103
Epoch 143/500
405/405 [==============================] - 0s 34us/sample - loss: 7.0318 - mae: 1.8926 - mse: 7.0318
Epoch 144/500
405/405 [==============================] - 0s 34us/sample - loss: 6.8554 - mae: 1.8601 - mse: 6.8554
Epoch 145/500
405/405 [==============================] - 0s 35us/sample - loss: 7.0176 - mae: 1.8735 - mse: 7.0176
Epoch 146/500
405/405 [==============================] - 0s 34us/sample - loss: 6.8861 - mae: 1.8578 - mse: 6.8861
Epoch 147/500
405/405 [==============================] - 0s 34us/sample - loss: 6.8609 - mae: 1.8742 - mse: 6.8609
Epoch 148/500
405/405 [==============================] - 0s 33us/sample - loss: 6.8952 - mae: 1.8727 - mse: 6.8952
Epoch 149/500
405/405 [==============================] - 0s 35us/sample - loss: 6.9020 - mae: 1.8823 - mse: 6.9020
Epoch 150/500
405/405 [==============================] - 0s 35us/sample - loss: 6.6972 - mae: 

405/405 [==============================] - 0s 36us/sample - loss: 5.1177 - mae: 1.6079 - mse: 5.1177
Epoch 214/500
405/405 [==============================] - 0s 35us/sample - loss: 5.1815 - mae: 1.6261 - mse: 5.1815
Epoch 215/500
405/405 [==============================] - 0s 35us/sample - loss: 5.0212 - mae: 1.5918 - mse: 5.0212
Epoch 216/500
405/405 [==============================] - 0s 35us/sample - loss: 5.2052 - mae: 1.6379 - mse: 5.2052
Epoch 217/500
405/405 [==============================] - 0s 35us/sample - loss: 5.1008 - mae: 1.6109 - mse: 5.1008
Epoch 218/500
405/405 [==============================] - 0s 41us/sample - loss: 5.1113 - mae: 1.6370 - mse: 5.1113
Epoch 219/500
405/405 [==============================] - 0s 40us/sample - loss: 4.9893 - mae: 1.6040 - mse: 4.9893
Epoch 220/500
405/405 [==============================] - 0s 36us/sample - loss: 5.0259 - mae: 1.5995 - mse: 5.0259
Epoch 221/500
405/405 [==============================] - 0s 36us/sample - loss: 5.0011 - mae: 

405/405 [==============================] - 0s 35us/sample - loss: 4.0961 - mae: 1.4547 - mse: 4.0961
Epoch 285/500
405/405 [==============================] - 0s 35us/sample - loss: 3.8877 - mae: 1.4056 - mse: 3.8877
Epoch 286/500
405/405 [==============================] - 0s 34us/sample - loss: 4.0902 - mae: 1.4444 - mse: 4.0902
Epoch 287/500
405/405 [==============================] - 0s 35us/sample - loss: 4.0106 - mae: 1.4176 - mse: 4.0106
Epoch 288/500
405/405 [==============================] - 0s 33us/sample - loss: 3.9139 - mae: 1.4156 - mse: 3.9139
Epoch 289/500
405/405 [==============================] - 0s 33us/sample - loss: 3.9209 - mae: 1.4293 - mse: 3.9209
Epoch 290/500
405/405 [==============================] - 0s 33us/sample - loss: 3.9830 - mae: 1.4270 - mse: 3.9830
Epoch 291/500
405/405 [==============================] - 0s 36us/sample - loss: 3.9043 - mae: 1.4034 - mse: 3.9043
Epoch 292/500
405/405 [==============================] - 0s 33us/sample - loss: 3.9682 - mae: 

405/405 [==============================] - 0s 34us/sample - loss: 3.1944 - mae: 1.2487 - mse: 3.1944
Epoch 356/500
405/405 [==============================] - 0s 34us/sample - loss: 3.2071 - mae: 1.2619 - mse: 3.2071
Epoch 357/500
405/405 [==============================] - 0s 33us/sample - loss: 3.2037 - mae: 1.2576 - mse: 3.2037
Epoch 358/500
405/405 [==============================] - 0s 33us/sample - loss: 3.2391 - mae: 1.2713 - mse: 3.2391
Epoch 359/500
405/405 [==============================] - 0s 35us/sample - loss: 3.1329 - mae: 1.2297 - mse: 3.1329
Epoch 360/500
405/405 [==============================] - 0s 34us/sample - loss: 3.2185 - mae: 1.2568 - mse: 3.2185
Epoch 361/500
405/405 [==============================] - 0s 34us/sample - loss: 3.0594 - mae: 1.2230 - mse: 3.0594
Epoch 362/500
405/405 [==============================] - 0s 33us/sample - loss: 3.1052 - mae: 1.2218 - mse: 3.1052
Epoch 363/500
405/405 [==============================] - 0s 34us/sample - loss: 3.1230 - mae: 

405/405 [==============================] - 0s 35us/sample - loss: 2.5544 - mae: 1.1211 - mse: 2.5544
Epoch 427/500
405/405 [==============================] - 0s 34us/sample - loss: 2.5700 - mae: 1.1279 - mse: 2.5700
Epoch 428/500
405/405 [==============================] - 0s 34us/sample - loss: 2.6258 - mae: 1.1606 - mse: 2.6258
Epoch 429/500
405/405 [==============================] - 0s 32us/sample - loss: 2.6867 - mae: 1.1638 - mse: 2.6867
Epoch 430/500
405/405 [==============================] - 0s 34us/sample - loss: 2.4882 - mae: 1.0937 - mse: 2.4882
Epoch 431/500
405/405 [==============================] - 0s 35us/sample - loss: 2.5283 - mae: 1.1518 - mse: 2.5283
Epoch 432/500
405/405 [==============================] - 0s 34us/sample - loss: 2.5025 - mae: 1.1187 - mse: 2.5025
Epoch 433/500
405/405 [==============================] - 0s 34us/sample - loss: 2.5344 - mae: 1.1084 - mse: 2.5344
Epoch 434/500
405/405 [==============================] - 0s 34us/sample - loss: 2.4555 - mae: 

405/405 [==============================] - 0s 32us/sample - loss: 2.1084 - mae: 1.0259 - mse: 2.1084
Epoch 498/500
405/405 [==============================] - 0s 33us/sample - loss: 2.1066 - mae: 1.0260 - mse: 2.1066
Epoch 499/500
405/405 [==============================] - 0s 33us/sample - loss: 2.1957 - mae: 1.0592 - mse: 2.1957
Epoch 500/500
405/405 [==============================] - 0s 33us/sample - loss: 2.1238 - mae: 1.0165 - mse: 2.1238


In [18]:
score = keras_model.predict(X)

In [19]:
score.shape

(405, 1)

In [20]:
loss, mae, mse = keras_model.evaluate(normed_test_data.values, test_labels.values, verbose=2)

print("Loss : {:5.2f}".format(loss))
print("Mae : {:5.2f}".format(mae))
print("Mse : {:5.2f}".format(mse))

101/1 - 0s - loss: 10.3712 - mae: 1.9117 - mse: 6.0931
Loss :  6.09
Mae :  1.91
Mse :  6.09


In [21]:
keras_model.save('../model/keras_model.h5')

In [22]:
print(keras_model.input_shape)

# get input, output node names for the TF graph from the Keras model
input_name = keras_model.inputs[0].name.split(':')[0]
keras_output_node_name = keras_model.outputs[0].name.split(':')[0]
graph_output_node_name = keras_output_node_name.split('/')[-1]

print(input_name)
print(keras_output_node_name)
print(graph_output_node_name)

core_model = tfcoreml.convert(tf_model_path='../model/keras_model.h5',
                         input_name_shape_dict={input_name: (1, 13)},
                         output_feature_names=[graph_output_node_name],
                         minimum_ios_deployment_target='13')
core_model.save('../model/keras_model.mlmodel')

core_model

(None, 13)
dense_input
dense_2/Identity
Identity
0 assert nodes deleted
['sequential/dense_2/MatMul/ReadVariableOp/resource:0', 'sequential/dense/MatMul/ReadVariableOp/resource:0', 'sequential/dense/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense_2/MatMul/ReadVariableOp:0', 'sequential/dense_2/BiasAdd/ReadVariableOp:0', 'sequential/dense/MatMul/ReadVariableOp:0', 'sequential/dense_1/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense_1/BiasAdd/ReadVariableOp:0', 'sequential/dense_2/BiasAdd/ReadVariableOp/resource:0', 'sequential/dense_1/MatMul/ReadVariableOp:0', 'sequential/dense/BiasAdd/ReadVariableOp:0', 'sequential/dense_1/MatMul/ReadVariableOp/resource:0']
6 nodes deleted
0 nodes deleted
0 nodes deleted
[Op Fusion] fuse_bias_add() deleted 6 nodes.
2 identity nodes deleted
3 disconnected nodes deleted
[SSAConverter] Converting function main ...
[SSAConverter] [1/6] Converting op type: 'Placeholder', name: 'dense_input', output_shape: (1, 13).
[SSAConverter] [2/6] Converting 

input {
  name: "dense_input"
  type {
    multiArrayType {
      shape: 1
      shape: 13
      dataType: DOUBLE
    }
  }
}
output {
  name: "Identity"
  type {
    multiArrayType {
      dataType: DOUBLE
    }
  }
}
metadata {
  userDefined {
    key: "coremltoolsVersion"
    value: "3.1"
  }
}

In [23]:
spec = core_model.get_spec()
spec

specificationVersion: 4
description {
  input {
    name: "dense_input"
    type {
      multiArrayType {
        shape: 1
        shape: 13
        dataType: DOUBLE
      }
    }
  }
  output {
    name: "Identity"
    type {
      multiArrayType {
        dataType: DOUBLE
      }
    }
  }
  metadata {
    userDefined {
      key: "coremltoolsVersion"
      value: "3.1"
    }
  }
}
neuralNetwork {
  layers {
    name: "sequential/dense/MatMul"
    input: "dense_input"
    output: "sequential/dense/MatMul"
    inputTensor {
      rank: 2
      dimValue: 1
      dimValue: 13
    }
    outputTensor {
      rank: 2
      dimValue: 1
      dimValue: 64
    }
    batchedMatmul {
      weightMatrixFirstDimension: 13
      weightMatrixSecondDimension: 64
      hasBias: true
      weights {
        floatValue: 0.2577735185623169
        floatValue: -0.08577174693346024
        floatValue: 0.4772864878177643
        floatValue: -0.11823222786188126
        floatValue: -0.5925122499465942
     